In [ ]:
# Mounting Google Colab with Google Drive

from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
# Moving to the directory of radialProfile.py so we can import it below

import os
os.chdir("/content/drive/My Drive/deepfake/Frequency/Faces-HQ")

In [ ]:
# Importing Other Required Modules

import cv2
import numpy as np
import radialProfile
import glob
from matplotlib import pyplot as plt
import pickle



In [ ]:
# load the model from disk. (Here, we are using svc model. Using linear reg. model is exactly similar)
loaded_model = pickle. load(open('finalized_svc_model_1.sav', 'rb'))

# loaded_model_lr = pickle. load(open('finalized_lr_model_1.sav', 'rb'))

In [ ]:
# Load the Cascade classifier to detect face from disk

classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:

video = cv2.VideoCapture('deepfake1.mkv')  # Load the video File

epsilon = 1e-8                         # A small constant Epsilon, used as in previous for preprocessing
prediction_list=[]                     # An empty list initialized to take the predictions from face of each frame
j=0                                    # Here, we are taking only first 100 frames, so initializing the variable for count

while (j<=100):
    j=j+1
    (f, im) = video.read() # f returns only True, False according to video access
   

    if f != True:         # If f is not true i.e., no frame is found, break the operation
       break


    # detectfaces using haarcascade_frontalface classifier
    faces = classifier.detectMultiScale(
        im, # stream 
        scaleFactor=1.10, # change these parameters to improve your video processing performance
        minNeighbors=20, 
        minSize=(30, 30) # min image detection size
        ) 
    
    # Draw rectangles around each face
    for (x, y, w, h) in faces:

        # Saving faces according to detected coordinates, preprocessing them as on training datas and feeding them to classifier.
        sub_face = im[y:y+h, x:x+w]
        sub_face = cv2.cvtColor(sub_face, cv2.COLOR_BGR2GRAY)      # Since we need Gray images, we convert the BGR frame to Gray {OpenCv reads RGB image as BGR}
        img = cv2.resize(sub_face, (1024, 1024))                   # We have trained on the image of 1024 * 1024 So, the image to be fed in classifier must be same.
        f = np.fft.fft2(img)                                       # Applying Fourier Transform
        fshift = np.fft.fftshift(f)                                # Shifting the Origin: On shifting the Origin, the low frequency components comes to center and High freq. component move towards the corner.

        fshift += epsilon                                          # Adding small constant Epsilon 

        magnitude_spectrum = 20*np.log(np.abs(fshift))             # Calculating Magnitude Spectrum

        
        # Calculate the azimuthally averaged 1D power spectrum
        psd1D = radialProfile.azimuthalAverage(magnitude_spectrum)
        
        var = psd1D.reshape(-1, 722)                              # Reshaping The data with empty label { As training data had label, it wants testing data to be of same shape}
                                                                  # The label will then be predicted by the classifier trained before.

        predicted = loaded_model.predict(var).astype(int)         # astype(int) changes the data to integer type
        pre = predicted.tolist()                                  # tolist() changes the values on var 'predicted' to a list
        prediction_list.append(pre)                               # Appending the values from var 'pre' the prediction_list {Empty list initialized before}
                                                                  # On appending for many frames, we get nested list here

video.release()         # Realeasing the video frames after completion





final_pred = []            # Initializing other empty list, final_pred to keep the final predicted value

def reemovNestings(l):            # This functions removes the nesting from the list. Since we have got a nested list before, we need to remove nestings.
    for i in l:
        if type(i) == list:
            reemovNestings(i)
        else:
            final_pred.append(i)

reemovNestings(prediction_list)     # Now, we get a list without the nestings




# For the final prediction, we count the vote on each frame and the maximum number of vote we get is the final prediction.
# i.e., If maximum frames are predicted fake, Final output is fake or vice versa.

def max_rep(final_pred):      # This Function is for counting the maximum repeated value in list.     
    counter = 0
    num = final_pred[0]
      
    for i in final_pred:
        curr_frequency = final_pred.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
  
    return num


a = max_rep(final_pred)       # Here, we get the maximum repeated prediction


# Now, Printing statement according to predicted value
if a == 0:
  print("The video is NOT from DEEPFAKE")

else:
    print("The video is from DEEPFAKE")


The video is from DEEPFAKE
